# Notebook [2]: Using the PDF converter



This notebook shows how to use the PDF converter to create an input dataframe for the cdQA pipeline from a directory of PDF files.


***Note:*** *To run this notebook you will need to have access to GPU. If you are using colab, you will need to install `cdQA` by executing `!pip install cdqa` in a cell.* 

In [1]:
!pip install cdqa

     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 286kB 8.1MB/s 
     |████████████████████████████████| 10.5MB 6.9MB/s 
     |████████████████████████████████| 317kB 56.1MB/s 
     |████████████████████████████████| 6.7MB 55.7MB/s 
     |████████████████████████████████| 748.9MB 22kB/s 
     |████████████████████████████████| 51kB 55kB/s 
     |████████████████████████████████| 860kB 47.1MB/s 
     |████████████████████████████████| 1.0MB 55.6MB/s 
  Created wheel for cdqa: filename=cdqa-1.3.9-cp36-none-any.whl size=47640 sha256=6fd61a7a0218f4916b03b4332a91bce7849932476e30279615296ef9aaa4766b
  Stored in directory: /root/.cache/pip/wheels/8b/9a/68/d3f7651ea29c30d1bebc9e946bf5a8cf922e1c86fb6b8a33d9
  Created wheel for tika: filename=tika-1.19-cp36-none-any.whl size=29224 sha256=c4e9662df7b638186fb32befb8053e3e1befd53c76fe4dc07284caa3160dc784
  Stored in directory: /root/.cache/pip/wheels/b4/db/8a/3a3f0c0725448eaa92703e3dda71e29dc13a119ff6c103

In [2]:
import os
import pandas as pd
from ast import literal_eval

from cdqa.utils.converters import pdf_converter
from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline import QAPipeline
from cdqa.utils.download import download_model

/usr/local/lib/python3.6/dist-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


### Download pre-trained reader model and PDF files

In [3]:
# Download model
download_model(model='bert-squad_1.1', dir='./models')

In [4]:
# Download pdf files from BNP Paribas public news
def download_pdf():
    import os
    import wget
    directory = './data/pdf/'
    models_url = [
      'https://invest.bnpparibas.com/documents/1q19-pr-12648',
      'https://invest.bnpparibas.com/documents/4q18-pr-18000',
      'https://invest.bnpparibas.com/documents/4q17-pr'
    ]

    print('\nDownloading PDF files...')

    if not os.path.exists(directory):
        os.makedirs(directory)
    for url in models_url:
        wget.download(url=url, out=directory)

download_pdf()

### Convert the PDF files into a DataFrame for cdQA pipeline

In [5]:
df = pdf_converter(directory_path='./data/pdf/')
df.head()

2019-12-27 19:11:04,422 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar to /tmp/tika-server.jar.
2019-12-27 19:11:04,971 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar.md5 to /tmp/tika-server.jar.md5.
2019-12-27 19:11:05,359 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


,title,paragraphs
0,4q17-pr,"[2017 FULL YEAR RESULTS PRESS RELEASE Paris,..."
1,4q18-pr2,"[2018 FULL YEAR RESULTS PRESS RELEASE Paris,..."
2,1q19-pr-12648,[FIRST QUARTER 2019 RESULTS PRESS RELEASE Pa...


### Instantiate the cdQA pipeline from a pre-trained reader model

In [6]:
cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib', max_df=1.0)

# Fit Retriever to documents
cdqa_pipeline.fit_retriever(df=df)

100%|██████████| 231508/231508 [00:00<00:00, 1245724.61B/s]


QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po..._size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=1.0, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

 ### Execute a query

In [0]:
query = 'How many contracts did BNP Paribas Cardif sell in 2019?'
prediction = cdqa_pipeline.predict(query)

### Explore predictions

In [8]:
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))
print('title: {}'.format(prediction[1]))
print('paragraph: {}'.format(prediction[2]))

query: How many contracts did BNP Paribas Cardif sell in 2019?
answer: 140,000
title: 1q19-pr-12648
paragraph: 3 Excluding PEL/CEL effects of +2 million euros compared to +1 million euros in the first quarter 2018    5 RESULTS AS AT 31 MARCH 2019   The new property and casualty offering launched in May 2018 as part of the partnership between  BNP Paribas Cardif and Matmut (Cardif IARD) recorded good growth with already  140,000 contracts sold as at 31 March 2019.   The business is accelerating individual customers’ mobile uses and developing self-care features with the roll-out of the conversational chatbots Telmi in the Mes comptes BNP Paribas app and Helloïz at Hello bank!.  
